## ここはいろいろ実験的なコードを書くところとする。

In [ ]:
import numpy as np
import pandas as pd
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')
train_goto = pd.read_csv("data/train_goto.tsv",sep='\t')
prod_goto = pd.read_csv("data/processed_train_goto.csv")

##### 6/22の作業

In [ ]:
df = pd.DataFrame()
df['pj']=prod_goto['pj_no']

In [ ]:
df['tc_mseki'] = prod_goto['tc_mseki']

In [ ]:
df[['tt_mseki', 'fukuin']]=prod_goto[['tt_mseki','fukuin']]

ここからモデル作成と予測をするコードを書く

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb 
from xgboost import XGBClassifier 

In [2]:
x_train = pd.read_csv('data/processed_train_goto.csv')
Y_train = pd.DataFrame(x_train['tc_tanka'])

In [3]:
X_train = x_train.drop(['id','pj_no','tc_tanka'],axis=1)
X_train = X_train.values
Y_train = Y_train.values.ravel()

In [ ]:
XGB = XGBClassifier(n_estimater=500,n_jobs=-1)
XGB.fit(X_train, Y_train)

In [ ]:
df.head()

#### 6/20の作業

In [ ]:
df = train_genba.copy()
df['gk_yoc_tm'] = df['gk_yoc_tm'].fillna(df['gk_yoc_tm'].mean())

In [ ]:
pd.DataFrame(prod_goto.isnull().sum()).to_csv("data/null_check.csv")

In [ ]:
import numpy as np
import pandas as pd
test_goto = pd.read_csv("data/processed_test_goto.csv")
pd.DataFrame(test_goto.isnull().sum()).to_csv("data/null_check_test.csv")

#### 確認の結果、次のカラムにNULLを含む
#### chiseki_kb_hb,kaoku_hb, mseki_rd_hb, mseki_dp_hb, chiseki_diff

In [ ]:
    # 間口(magutchi)
    # 幅員と同じように補完する。
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

#### 6/18,19の作業

In [ ]:
df = train_goto.copy()

In [ ]:
genba = pd.read_csv("data/processed_train_genba.csv")

In [ ]:
newdf=df.merge(genba, how="left", on='pj_no')

In [ ]:
newdf.to_csv("data/joined_data.csv")
df = newdf

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)
for i in range(len(df)):
    if ( df.at[i,'fukuin'] ==0 ) :
        df.at[i,'fukuin'] = df.at[i,'road1_fi']

In [ ]:
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

In [ ]:
df['setsudo_hi' ] = df['setsudo_hi'].fillna('unknown')
for i in range(len(df)):
    if ( df.at[i,'setsudo_hi']=='unknown'):
        df.at[i.'setsudo_hi']=df.at[i,'road1_hk']

In [ ]:
df.head()

In [ ]:
help(list.append)

In [ ]:
df = pd.read_csv('data/processed_train_goto.csv')

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.loc[146,'fukuin']

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)

In [ ]:
df['fukuin'].isnull().sum()

#### いつのだかわからないコード

In [ ]:
df = train_genba.copy()

In [ ]:
df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)

In [ ]:
df.loc[0:20,['bas_toho1','eki_kyori1','teiho1','bus','walk']]

#### 下記は6/16夜の作業

In [ ]:
def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])
    

#### 下記は6/16の作業

In [ ]:
df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)

In [ ]:
np.isnan(df.loc[0,'yoto2'])

In [ ]:
type(df.loc[26,'yoto2'])==str

In [ ]:
df.loc[20,'shu_soon']

In [ ]:
df['shu_soon'].map({'○':1, np.NaN:0})

In [ ]:
def select_yoto(x):
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    """
    if ( np.isnan(x['yoto2'])):
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    """
    """"
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    """
    return pd.Series([yoto, kempei, yoseki])


#### 下記は6/15の作業

In [ ]:
df = df.drop('jukyo',axis=1)

In [ ]:
# df['yoto'], df['kempei'],df['yoseki']= df.apply(select_yoto,axis=1)
df[['yoto','kempei','yoseki']] = df.apply(select_yoto,axis=1)

In [ ]:
df.head()

In [ ]:
org_df = df.copy()

In [ ]:
df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)

In [ ]:
df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)

In [ ]:
df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)

In [ ]:
df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']

In [ ]:
a = pd.get_dummies(df[['usui']],drop_first=False)

In [ ]:
df2 = pd.merge(df, a, left_index=True, right_index=True)

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df = df.fillna({'tateuri_su': 0})

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_hk']!=np.nan else 3)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)

In [ ]:
import math

In [ ]:
df['road_num']=df.apply(lambda x: 4 if not math.isnan(x['road4_fi']) else 3 if not math.isnan(x['road3_fi']) else 2 if not math.isnan(x['road2_fi']) else 1, axis=1)

In [ ]:
df['toshikuiki']=df['toshikuiki2'].fillna('empty')

In [ ]:
df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)

In [ ]:
df['toshikuiki']

In [ ]:
df.loc[1549,'toshikuiki2']

In [ ]:
def select_yoto(x):
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    return pd.Series([yoto, kempei, yoseki])
